In [4]:
import os
import pandas
import numpy
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from tqdm import tqdm
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [2]:
dataDir = '/home/joel/Documents/KaggleComps/SeedlingId/'
trainDir = os.path.join(dataDir, 'train')
testDir = os.path.join(dataDir, 'test')
sampleSub = pandas.read_csv(os.path.join(dataDir, 'sample_submission.csv'))

**Exploration**

In [122]:
sampleSub.head()

,file,species
0,0021e90e4.png,Sugar beet
1,003d61042.png,Sugar beet
2,007b3da8b.png,Sugar beet
3,0086a6340.png,Sugar beet
4,00c47e980.png,Sugar beet


In [13]:
categories = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
weeds = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Fat Hen', 'Loose Silky-bent',
         'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill']
crops = ['Common wheat', 'Maize', 'Sugar beet']
for cat in categories:
    print('There are {} pictures of {}'.format(len(os.listdir(os.path.join(dataDir,'trainVal' , cat))), cat))

There are 263 pictures of Black-grass
There are 390 pictures of Charlock
There are 287 pictures of Cleavers
There are 611 pictures of Common Chickweed
There are 221 pictures of Common wheat
There are 475 pictures of Fat Hen
There are 654 pictures of Loose Silky-bent
There are 221 pictures of Maize
There are 516 pictures of Scentless Mayweed
There are 231 pictures of Shepherds Purse
There are 496 pictures of Small-flowered Cranesbill
There are 385 pictures of Sugar beet


**Pre-Processing**

In [14]:
minNumSamples = 221
trainVal = []
catindex=0
for cat in categories:
    for file in os.listdir(os.path.join(trainDir, cat))[0:minNumSamples]:
        trainVal.append(['train/{}/{}'.format(cat, file), cat, catindex])
    catindex+=1
    
trainVal = pandas.DataFrame(trainVal, columns=["file", "category", "category_index"])
trainVal.shape

In [5]:
trainVal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2652 entries, 0 to 2651
Data columns (total 3 columns):
file              2652 non-null object
category          2652 non-null object
category_index    2652 non-null int64
dtypes: int64(1), object(2)
memory usage: 62.2+ KB


In [6]:
test=[]
for file in os.listdir(testDir):
    test.append(['test/{}'.format(file), file])
test = pandas.DataFrame(test, columns=["path", "file"])
test.shape

(794, 2)

In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 794 entries, 0 to 793
Data columns (total 2 columns):
path    794 non-null object
file    794 non-null object
dtypes: object(2)
memory usage: 12.5+ KB


In [5]:
mask = numpy.random.rand(len(trainVal)) < 0.8
train = trainVal[mask]
validate = trainVal[~mask]
train.shape

(2123, 3)

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2138 entries, 1 to 2651
Data columns (total 3 columns):
file              2138 non-null object
category          2138 non-null object
category_index    2138 non-null int64
dtypes: int64(1), object(2)
memory usage: 66.8+ KB


In [12]:
validate.shape

(514, 3)

In [11]:
validate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 549 entries, 1 to 2646
Data columns (total 3 columns):
file              549 non-null object
category          549 non-null object
category_index    549 non-null int64
dtypes: int64(1), object(2)
memory usage: 17.2+ KB


*Image Augmentation*

In [7]:
trainGenerator=image.ImageDataGenerator(
        shear_range=0.2, zoom_range=0.2,
        horizontal_flip=True)

def maketensor(path):
    img = image.load_img(path, target_size=(299, 299))
    x = image.img_to_array(img)
    return numpy.expand_dims(x, axis=0)
def maketensors(paths):
    tensorlist = [maketensor(path) for path in tqdm(paths)]
    return numpy.vstack(tensorlist)
testTensors = maketensors(test['path']).astype('float32')/255

In [9]:
testTensors = maketensors(test['path']).astype('float32')/255
i = 0
for batch in trainGenerator.flow(testTensors, batch_size=1,
                          save_to_dir='preview', save_prefix='seedling', save_format='jpeg'):
    i += 1
    if i > 20:
        break

100%|██████████| 794/794 [00:03<00:00, 242.83it/s]


In [ ]:
trainTensors = maketensors(train['file']).astype('float32')/255
validationTensors = maketensors(validate['file']).astype('float32')/255
testTensors = maketensors(test['path']).astype('float32')/255

In [14]:
trainTargets = np_utils.to_categorical(numpy.array(train['category_index']), 12)
validationTargets = np_utils.to_categorical(numpy.array(validate['category_index']), 12)


**Baseline scratchbuilt CNN**

In [57]:
model = Sequential()
model.add(Conv2D(16, 3, padding='same', input_shape=(224,224,3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(12))
model.add(Activation('softmax'))

model.summary()
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_204 (Conv2D)          (None, 224, 224, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
conv2d_205 (Conv2D)          (None, 112, 112, 32)      4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_206 (Conv2D)          (None, 56, 56, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 50176)             0         
__________

In [221]:
checkpointer = ModelCheckpoint(filepath='weights/baselineWithAugmentation.hdf5', verbose=1, save_best_only=True)

model.fit_generator(trainGenerator.flow(trainTensors, trainTargets, batch_size=20),
                    steps_per_epoch=len(trainTensors) / 20, epochs=20,
                    validation_data=(validationTensors, validationTargets), callbacks=[checkpointer], verbose=1)

Epoch 1/20
108/107 [==============================] - 137s 1s/step - loss: 1.9766 - acc: 0.3052 - val_loss: 1.5247 - val_acc: 0.4889
Epoch 2/20
108/107 [==============================] - 137s 1s/step - loss: 1.4330 - acc: 0.5023 - val_loss: 1.6065 - val_acc: 0.4101
Epoch 3/20
108/107 [==============================] - 138s 1s/step - loss: 1.2363 - acc: 0.5765 - val_loss: 1.6244 - val_acc: 0.4384
Epoch 4/20
108/107 [==============================] - 138s 1s/step - loss: 1.1006 - acc: 0.6271 - val_loss: 1.1616 - val_acc: 0.6141
Epoch 5/20
108/107 [==============================] - 136s 1s/step - loss: 1.0145 - acc: 0.6650 - val_loss: 1.0304 - val_acc: 0.6606
Epoch 6/20
108/107 [==============================] - 136s 1s/step - loss: 0.9177 - acc: 0.6876 - val_loss: 1.0623 - val_acc: 0.6747
Epoch 7/20
108/107 [==============================] - 136s 1s/step - loss: 0.8260 - acc: 0.7210 - val_loss: 1.2848 - val_acc: 0.6364
Epoch 8/20
108/107 [==============================] - 136s 1s/step - 

In [239]:
model.load_weights('weights/baselineWithAugmentation.hdf5')
seedlingIndexes = [numpy.argmax(model.predict(numpy.expand_dims(tensor, axis=0))) for tensor in testTensors]
seedlingPredictions=[categories[index] for index in seedlingIndexes]
submission=pandas.DataFrame({'file':test['file'],'species': seedlingPredictions}, columns=["file", "species"])
submission.to_csv('/home/joel/Documents/KaggleComps/SeedlingId/baselineSubmission.csv')

Model got a mean f-score of 0.801 against test data on kaggle leaderboard.

**From bottleneck features of InceptionResNetv2**

In [8]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import Model

In [ ]:
IRmodel = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None, input_shape=(299, 299, 3))


In [22]:
IRmodel.summary()
IRmodel.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_4[0][0]                   
__________________________________________________________________________________________________
activation_2 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_5 (

In [ ]:
bottleneckFeatures = IRmodel.predict(trainTensors)
validationFeatures = IRmodel.predict(validationTensors)

In [ ]:
testFeatures = IRmodel.predict(testTensors)

In [269]:
Dmodel = Sequential()
Dmodel.add(GlobalAveragePooling2D(input_shape=(8, 8, 1536)))
Dmodel.add(Dense(122))
Dmodel.add(Activation('tanh'))
Dmodel.add(Dropout(.5))
Dmodel.add(Dense(12))
Dmodel.add(Activation('softmax'))

Dmodel.summary()
Dmodel.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_78  (None, 1536)              0         
_________________________________________________________________
dense_187 (Dense)            (None, 122)               187514    
_________________________________________________________________
activation_363 (Activation)  (None, 122)               0         
_________________________________________________________________
dropout_81 (Dropout)         (None, 122)               0         
_________________________________________________________________
dense_188 (Dense)            (None, 12)                1476      
_________________________________________________________________
activation_364 (Activation)  (None, 12)                0         
Total params: 188,990
Trainable params: 188,990
Non-trainable params: 0
_________________________________________________________________


In [270]:
checkpointer = ModelCheckpoint(filepath='weights/FirstModel.hdf5', verbose=1, save_best_only=True)

Dmodel.fit(bottleneckFeatures, trainTargets, epochs=25, batch_size=25,
                    validation_data=(validationFeatures, validationTargets), callbacks=[checkpointer], verbose=1)

Train on 2128 samples, validate on 524 samples
Epoch 1/25
2128/2128 [==============================] - 5s 3ms/step - loss: 2.0131 - acc: 0.3590 - val_loss: 1.7136 - val_acc: 0.4103
Epoch 2/25
2128/2128 [==============================] - 1s 372us/step - loss: 1.2988 - acc: 0.5611 - val_loss: 1.1639 - val_acc: 0.5897
Epoch 3/25
2128/2128 [==============================] - 1s 380us/step - loss: 1.0379 - acc: 0.6391 - val_loss: 1.1992 - val_acc: 0.5630
Epoch 4/25
2128/2128 [==============================] - 1s 370us/step - loss: 0.9291 - acc: 0.6631 - val_loss: 1.3350 - val_acc: 0.5782
Epoch 5/25
2128/2128 [==============================] - 1s 370us/step - loss: 0.8120 - acc: 0.7138 - val_loss: 0.9859 - val_acc: 0.6412
Epoch 6/25
2128/2128 [==============================] - 1s 382us/step - loss: 0.7375 - acc: 0.7439 - val_loss: 1.3860 - val_acc: 0.5115
Epoch 7/25
2128/2128 [==============================] - 1s 381us/step - loss: 0.6735 - acc: 0.7514 - val_loss: 0.8851 - val_acc: 0.7099
Epo

In [274]:
Dmodel.load_weights('weights/FirstModel.hdf5')
seedlingIndexes = [numpy.argmax(Dmodel.predict(numpy.expand_dims(tensor, axis=0))) for tensor in testFeatures]
seedlingPredictions=[categories[index] for index in seedlingIndexes]
FirstModelSubmission=pandas.DataFrame({'file':test['file'],'species': seedlingPredictions}, columns=["file", "species"])
FirstModelSubmission.to_csv('/home/joel/Documents/KaggleComps/SeedlingId/FirstModelSubmission.csv')

This model got an mean f-score of .84 on the kaggle leaderboard test data.

**Implement image augmentation and model fine-tuning for transfer learning**

In [9]:
FTmodel = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=None, input_shape=(299, 299, 3))
FTmodel.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

__________________________________________________________________________________________________
conv2d_186 (Conv2D)             (None, 8, 8, 224)    129024      activation_185[0][0]             
__________________________________________________________________________________________________
batch_normalization_186 (BatchN (None, 8, 8, 224)    672         conv2d_186[0][0]                 
__________________________________________________________________________________________________
activation_186 (Activation)     (None, 8, 8, 224)    0           batch_normalization_186[0][0]    
__________________________________________________________________________________________________
conv2d_184 (Conv2D)             (None, 8, 8, 192)    399360      block8_5_ac[0][0]                
__________________________________________________________________________________________________
conv2d_187 (Conv2D)             (None, 8, 8, 256)    172032      activation_186[0][0]             
__________

In [10]:
for layer in FTmodel.layers:
    layer.trainable=False

x=FTmodel.output
x = GlobalAveragePooling2D()(x)
x = Dense(122, activation='tanh')(x)
predictions = Dense(12, activation='softmax')(x)

untunedmodel = Model(inputs=FTmodel.input, outputs=predictions)

untunedmodel.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

Train new fully connected segment. (CNN frozen)

In [ ]:
checkpointer = ModelCheckpoint(filepath='weights/UnTunedModel.hdf5', verbose=1, save_best_only=True)

untunedmodel.fit_generator(trainGenerator.flow(trainTensors, trainTargets, batch_size=20),
                    steps_per_epoch=len(trainTensors) / 20, epochs=12,
                    validation_data=(validationTensors, validationTargets), callbacks=[checkpointer], verbose=1)

Fine-Tune CNN

In [13]:
for i, layer in enumerate(FTmodel.layers):
   print(i, layer.name)

0 input_1
1 conv2d_1
2 batch_normalization_1
3 activation_1
4 conv2d_2
5 batch_normalization_2
6 activation_2
7 conv2d_3
8 batch_normalization_3
9 activation_3
10 max_pooling2d_1
11 conv2d_4
12 batch_normalization_4
13 activation_4
14 conv2d_5
15 batch_normalization_5
16 activation_5
17 max_pooling2d_2
18 conv2d_9
19 batch_normalization_9
20 activation_9
21 conv2d_7
22 conv2d_10
23 batch_normalization_7
24 batch_normalization_10
25 activation_7
26 activation_10
27 average_pooling2d_1
28 conv2d_6
29 conv2d_8
30 conv2d_11
31 conv2d_12
32 batch_normalization_6
33 batch_normalization_8
34 batch_normalization_11
35 batch_normalization_12
36 activation_6
37 activation_8
38 activation_11
39 activation_12
40 mixed_5b
41 conv2d_16
42 batch_normalization_16
43 activation_16
44 conv2d_14
45 conv2d_17
46 batch_normalization_14
47 batch_normalization_17
48 activation_14
49 activation_17
50 conv2d_13
51 conv2d_15
52 conv2d_18
53 batch_normalization_13
54 batch_normalization_15
55 batch_normalization

In [18]:
untunedmodel.load_weights('weights/TunedModel.hdf5')

for layer in untunedmodel.layers[:618]:
   layer.trainable = False
for layer in untunedmodel.layers[618:]:
   layer.trainable = True

from keras.optimizers import SGD
untunedmodel.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
checkpointer = ModelCheckpoint(filepath='weights/TunedModel.hdf5', verbose=1, save_best_only=True)

untunedmodel.fit_generator(trainGenerator.flow(trainTensors, trainTargets, batch_size=20),
                    steps_per_epoch=len(trainTensors) / 20, epochs=12,
                    validation_data=(validationTensors, validationTargets), callbacks=[checkpointer], verbose=1)

Epoch 1/12
106/105 [==============================] - 3331s 31s/step - loss: 0.4162 - acc: 0.8588 - val_loss: 0.4681 - val_acc: 0.8675
Epoch 2/12
106/105 [==============================] - 3284s 31s/step - loss: 0.4014 - acc: 0.8619 - val_loss: 0.4479 - val_acc: 0.8713
Epoch 3/12
106/105 [==============================] - 3273s 31s/step - loss: 0.3985 - acc: 0.8647 - val_loss: 0.4461 - val_acc: 0.8731
Epoch 4/12
106/105 [==============================] - 3273s 31s/step - loss: 0.3842 - acc: 0.8733 - val_loss: 0.4404 - val_acc: 0.8713
Epoch 5/12
106/105 [==============================] - 3267s 31s/step - loss: 0.3728 - acc: 0.8771 - val_loss: 0.4453 - val_acc: 0.8713
Epoch 6/12
106/105 [==============================] - 3283s 31s/step - loss: 0.3563 - acc: 0.8700 - val_loss: 0.4363 - val_acc: 0.8731
Epoch 7/12
106/105 [==============================] - 3267s 31s/step - loss: 0.3587 - acc: 0.8767 - val_loss: 0.4242 - val_acc: 0.8750
Epoch 8/12
106/105 [==============================] - 3

In [23]:
untunedmodel.load_weights('weights/TunedModel.hdf5')
seedlingIndexes = [numpy.argmax(untunedmodel.predict(numpy.expand_dims(tensor, axis=0))) for tensor in testTensors]
seedlingPredictions=[categories[index] for index in seedlingIndexes]
tunedModelSubmission=pandas.DataFrame({'file':test['file'],'species': seedlingPredictions}, columns=["file", "species"])
tunedModelSubmission.to_csv('/home/joel/Documents/KaggleComps/SeedlingId/tunedModelSubmission.csv')

The fine-tuned model with image augmentation got a mean f-score of 0.8942 on the kaggle leaderboard test data.

**In the wild data**

In [55]:
Wild = []
catindex = 0
for cat in os.listdir(os.path.join(dataDir, 'InTheWild')):
    for file in os.listdir(os.path.join(dataDir, 'InTheWild', cat)):
        Wild.append(['InTheWild/{}/{}'.format(cat,file), file, cat, catindex])
    catindex+=1
InTheWild = pandas.DataFrame(Wild, columns=["path", "file", "category", "catindex"])
InTheWild.shape
wildTargets = np_utils.to_categorical(numpy.array(InTheWild['catindex']), 12)

In [60]:
wildTensors = maketensors(InTheWild['path']).astype('float32')/255

100%|██████████| 96/96 [00:00<00:00, 345.55it/s]


In [56]:
untunedmodel.load_weights('weights/TunedModel.hdf5')
untunedmodel.evaluate(x=wildTensors, y=wildTargets, verbose=1)

96/96 [==============================] - 112s 1s/step


[4.3628765741984052, 0.15625]

In [61]:
model.load_weights('weights/baselineWithAugmentation.hdf5')
model.evaluate(x=wildTensors, y=wildTargets, verbose=1)

96/96 [==============================] - 2s 23ms/step


[9.8399426142374669, 0.16666666666666666]

**Weed/not weed accuracy**

In [11]:

validationTensors = maketensors(validate['file']).astype('float32')/255

100%|██████████| 529/529 [00:05<00:00, 88.26it/s]


In [12]:
Weed=[]
valcats=numpy.array(validate['category'])
untunedmodel.load_weights('weights/TunedModel.hdf5')
seedlingIndexes = [numpy.argmax(untunedmodel.predict(numpy.expand_dims(tensor, axis=0))) for tensor in validationTensors]
seedlingPredictions=[categories[index] for index in seedlingIndexes]
for item in seedlingPredictions:
    if item in weeds:
        Weed.append(True)
    else:
        Weed.append(False)

In [13]:
ValWeed = []
for item in valcats:
    if item in weeds:
        ValWeed.append(True)
    else:
        ValWeed.append(False)
count=0
correct=0
for guess in Weed:
    if guess == ValWeed[count]:
        correct+=1
    else:
        print("Mis-identified ", valcats[count], " as ", seedlingPredictions[count], ".\n")
    count+=1
percent=correct/count
print("Guessed ", percent, "% correct.")

Mis-identified  Black-grass  as  Sugar beet .

Mis-identified  Black-grass  as  Common wheat .

Mis-identified  Cleavers  as  Sugar beet .

Mis-identified  Common wheat  as  Common Chickweed .

Mis-identified  Fat Hen  as  Sugar beet .

Mis-identified  Sugar beet  as  Common Chickweed .

Mis-identified  Sugar beet  as  Fat Hen .

Mis-identified  Sugar beet  as  Fat Hen .

Mis-identified  Sugar beet  as  Common Chickweed .

Guessed  0.9829867674858223 % correct.
